# 챕터 19: 동적 속성과 프로퍼티

**모듈이 제공하는 모든 서비스는 통일된 표기법을 이용해서 접근할 수 있어야 한다.통일된 표기법은 저장소를 이용해서 구현하거나 계산을 통해 구현하는 경우에도 모두 동일하게 적용된다.**

## 19.1: 동적 속성을 이용한 데이터 랭글링

In [1]:
# 예제 19.1: osconfeed.json에서 가져온 샘플 레코드. 몇몇 필드 내용은 생략했다.
{"Schedule":
    {"conferences": [{"serial": 115}],
    "events": [
        {"serial": 34505,
        "name": "Why Schools Don't Use Open Source to Tech Programming",
        "event_type": "40-minute conference session",
        "time_start": "2014-07-23 11:30:00",
        "time_stop": "2014-07-23 12:10:00",
        "venue_serial": 1462,
        "description": "Aside from the fact that high school programming...",
        "website_url": "http://oscon.com/public/schedule/detail/34505",
        "speakers": [157509],
        "categories": ["Education"]}
    ],
    "speakers": [
        {"serial": 157509,
        "name": "Robert Lefkowitz",
        "photo": null,
        "url": "http://sharewave.com/",
        "position": "CTO",
        "affiliation": "Sharewave",
        "twitter": "sharewaveteam",
        "bio": "Robert 'r0ml' Lefkowitz is the CTO at Sharewave, a startup..."}
    ],
    "venues": [
        {"serial": 1462,
        "name": "F151",
        "category": "Conference Venues"}
    ]}}

NameError: name 'null' is not defined

In [ ]:
# 예제 19-2: osconfeed.py: osconfeed.json 내려받기, doctest는 [예제 19-3]에 있다.
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)
        with urlopen(URL) as remote, open(JSON, 'wb') as local:
            local.write(remote.read())
    with open(JSON) as fp:
        return json.load(fp)

In [ ]:
# 예제 19-3: osconfeed.py: [예제 19-2]에 대한 doctest
feed = load()
sorted(feed['Schedule'].keys())
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))

In [ ]:
# 동적 속성을 이용해서 JSON과 유사한 데이터 둘러보기
from collections import abc

class FrozenJSON:
    """점 표기법을 이용해서 JSON과 유사한 객체를 순화하는
        읽기 전용 퍼사드 클래스"""
    
    def __init__(self, mapping):
        self.__data = dict(mapping)
    
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])
    
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

In [ ]:
# __new__()을 이용한 융통성 있는 객체 생성
# 객체를 생성하는 의사코드
def object_maker(the_class, some_arg):
    new_object = the_class.__new__(some_arg)
    if isinstance(new_object, the_class):
        the_class.__init__(new_object, some_arg)
    return new_object

# 다음 두 문장은 거의 동일하다.
x = Foo('bar')
x = object_maker(Foo, 'bar')

In [ ]:
# 예제 19-7: explore2.py: FrozenJSON 객체든 아니든 새로운 객체를 생성하는 대신 __new__() 사용하기
from collections import abc
from keyword import iskeyword

class FrozenJSON:
    """점 표기법을 이용해서 JSON과 유사한 객체를 둘러보기 위한 
        읽기 전용 퍼사드 클래스"""
    
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg
        
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping_items():
            if iskeyword(key):
                key += '_'
            self.__data[key] = value
    
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])

In [ ]:
# shelve를 이용해서 OSCON 피드 구조 변경하기
# 예제 19-9: schedule1.py: shelve.Shelf에 저장된 OSCON 일정 데이터 둘러보기

import warnings

import osconfeed

DB_NAME = 'data/schedule1_db'
CONFERENCE = 'conference.115'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
def load_db(db):
    raw_data = osconfeed.load()
    warnings.warn('loading '+DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collections[:-1]
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = Record(**record)

### 프로퍼티를 이용해서 연결된 레코드 읽기

* Record 클래스

* DbRecord 클래스: Record의 서브클래스.

* Event 클래스: DbRecord의 서브클래스.

In [ ]:
# 예제 19-11: schedule2.py: 임포트, 상수. 개선된 Record 클래스.
import warnings
import inspect
import osconfeed

DB_NAME = 'data/schedule2_db'
CONFERENCE = 'conference.115'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
    def __eq__(self, other):
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

In [ ]:
# 예제 19-12: schedule2.py: MissingDatabaseError와 DbRecord 클래스
class MissingDatabaseError(RuntimeError):
    """필요한 데이터베이스가 설정되어 있지 않을 때 발생한다."""
    
class DbRecord(Record):
    
    __db = None
    
    @staticmethod
    def set_db(db):
        DbRecord.__db = db
        
    @staticmethod
    def get_db():
        return DbRecord.__db
    
    @classmethod
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident]
        except TypeError:
            if db is None:
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else:
                raise
    
    def __repr__(self):
        if hasattr(self, 'serial'):
            cls_name = self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name, self.serial)
        else:
            return super().__repr__()

In [ ]:
# 예제 19-14: schedule2.py: load_db() 함수
def load_db(db):
    raw_data = osconfeed.load()
    warnings.warn('loading' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collections[:-1]
        cls_name = record_type.capitalize()
        cls = globals().get(cls_name, DbRecord)
        if inspect.isclass(cls) and issubclass(cls, DbRecord):
            factory = cls
        else:
            factory = DbRecord
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = factory(**record)
        

## 19.2 속성을 검증하기 위해 프로퍼티 사용하기


In [2]:
# LineItem 버전 #1: 주문 항목 클래스
# bulkfood_v1.py: 기본적인 LineItem 클래스

class LineItem:
    
    def __init__(self, description, weight, price):
        self.describtion = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weigth * slef.price

In [ ]:
# LineItem 버전 #2: 검증하는 프로퍼티
# 예제 19-17: bulkfood_v2.py: weight 프로퍼티를 가진 LineItem
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):
        return self.__weight
    
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

## 19.3 프로퍼티 제대로 알아보기

In [ ]:
# 예제 19-18: bulkfood_v2.py: [예제 19-17]과 동일하지만 데커레이터를 사용하지 않는 LineItem

class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.__weight
    
    def set_weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')
            
    weight = property(get_weight, set_weight)

In [ ]:
# 프로퍼티 문서화

# 예제 19-22: 프로퍼티에 대한 문서화
class Foo:
    
    @property
    def bar(self):
        '''The bar attribute'''
        return self.__dict__['bar']
    
    @bar.setter
    def bar(self, value):
        self.__dict__['bar'] = value

## 19.4: 프로퍼티 팩토리 구현하기

In [ ]:
# 19-23: bulkfood_v2prop.py: quantity() 프로퍼티 팩토리 사용하기
class LineItem:
    weight = quantity('weight')
    price = quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [ ]:
# 예제 19-24: bulkfood_v2prop.py: quantity() 프로퍼티 팩토리
def quantity(storage_name):
    
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
            
    return property(qty_getter, qty_setter)

## 19.5 속성 제거 처리하기

In [ ]:
# 예제 19-26: blackknight.py:'몬티 파이튼과 성배'의 흑기사에서 영감을 얻었다.
class BlackKnight:
    
    def __init__(self):
        self.members = ['an arm', 'another arm', 'a leg', 'another leg']
        self.phrases = ["'Tis but a scratch",
                       "It's just a flesh wound.",
                       "I'm invinvible!",
                       "All right, we'ii call it a draw."]
    
    @property
    def member(self):
        print('next member is:')
        return self.members[0]
    
    @member.deleter
    def member(self):
        text = 'BLACK KNIGHT (loses {})\n-- {}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))

## 19.6 속성을 처리하는 핵심 속성 및 함수

* 속성 처리에 영향을 주는 특별 속성
    * _class_: 객체의 클래스에 대한 참조. 파이썬은 _getattr_()과 같은 특별 메서드를 객체 자체가 아니라 객체의 클래스에서만 검색한다.
    * _dict_: 객체나 클래스의 쓰기 가능 속성을 저장하는 매핑. _dict_를 가진 객체는 임의의 새로운 속성을 언제든지 설정할 수 있다. 클래스에 _slots_속성이 있자면, 이 클래스의 객체에서는 _dict_가 없을 수도 있다. 다음의 _slots_설명을 참조하라.
    * _slots_: 자신의 객체가 가질 수 있는 속성을 제한하려는 클래스에 정의하는 속성. _slots_는 허용돈 속성명을 담은 일종의 튜플이다. _dict_가 _slots_에 들어 있지 않으면, 이 클래스의 객체는 자체적인 _dict_를 가질 수 없고, 여기에 나열된 속성만 만들 수 있다.
    

* 속성을 처리하는 내장 함수
    * dir([object]): 대부분의 객체 속성을 나열한다.
    * getattr(object, name[, default]): object에서 name 문자열로 식별된 속성을 가져온다. 객체의 클래스나 슈퍼클래스에서 속성을 가져올 수 있다. 이러한 속성이 존재하지 않으면 getattr()은 AttributeError를 발생시키거나 default값을 반환한다.
    * hasattr(object, name): 해당 이름의 속성이 object에 있거나 상속 등의 메커니즘으로 가져올 수 있으면 True를 반환한다.
    * setattr(object, name, value): object가 허용하면 name 속성에 value를 할당한다.
    * vars([object]): object의 _dict_를 반환한다. dir() 메서드와 달리 _slots_는 있고 _dict_는 없는 클래스의 객체는 처리할 수 없다. 인수를 전달하지 않으면 vars()는 현재 범위의 _dict_를 가져오므로 locals()와 동일하게 작동한다.
    
* 속성을 처리하는 특별 메서드
    * _delattr_(self, name): del문을 이용해서 속성을 제거하려 할 때 호출된다. 즉 del obj.attr은 Class._delattr_(obj, 'attr')을 호출한다.
    * _dir_(self): 속성을 나열하기 위해 객체에 dir()을 호출할 때 호출된다. 즉, dir(obj)는 Class._dir_(obj)를 호출한다.
    * _getatt_(self, name): obj, Class, Class의 슈퍼클래스를 검색해서 명명된 속성을 가져오려고 시도하다 실패할 때 호출된다.
    * _getattribute_(self, name): 특별 속성이나 메서드가 아닌 속성을 가져올 때 언제나 호출된다.
    * _setattr_(self, name, value): 지명된 속성에 값을 설정할 때 언제나 호출된다.